# Tagmatic Tutorial: Complete Guide to Text Classification

Welcome to the comprehensive tutorial for **Tagmatic** - a flexible, user-defined text classification library using Large Language Models (LLMs).

## What You'll Learn

1. **Basic Usage** - Simple text classification with custom categories
2. **Sentiment Analysis** - Real-world example with customer feedback
3. **Topic Classification** - Categorizing news articles and content
4. **Content Moderation** - Detecting spam and inappropriate content
5. **Voting Classifier** - Improving accuracy with ensemble methods
6. **Advanced Features** - Batch processing, custom prompts, and configuration
7. **Best Practices** - Tips for optimal performance and accuracy

## Prerequisites

- Python 3.8+
- OpenAI API key (or other LLM provider)
- Basic understanding of text classification concepts

Let's get started! 🚀

## Setup and Installation

First, let's install the required dependencies and set up our environment.

In [1]:
# !pip install tagmatic langchain-openai python-dotenv

import os
from dotenv import load_dotenv
import warnings
warnings.filterwarnings('ignore')

# Load environment variables
load_dotenv()

# Import Tagmatic components
from tagmatic import Category, CategorySet, Classifier
from langchain_openai import ChatOpenAI

# Set up OpenAI API key
if not os.getenv("OPENAI_API_KEY"):
    print("⚠️  Please set your OPENAI_API_KEY environment variable")
    print("You can do this by creating a .env file with: OPENAI_API_KEY=your_key_here")
else:
    print("✅ OpenAI API key found!")

✅ OpenAI API key found!


## 1. Basic Usage: Getting Started with Tagmatic

Let's start with a simple example to understand the core concepts of Tagmatic.

In [2]:
# Step 1: Define your categories
basic_categories = CategorySet(categories=[
    Category(
        name="positive",
        description="Text expressing positive emotions, satisfaction, or happiness",
        examples=["I love this!", "Great job!", "This is amazing!"]
    ),
    Category(
        name="negative", 
        description="Text expressing negative emotions, complaints, or dissatisfaction",
        examples=["I hate this", "This is terrible", "Very disappointed"]
    ),
    Category(
        name="neutral",
        description="Text that is factual or doesn't express strong emotions",
        examples=["The meeting is at 3 PM", "It's 72 degrees outside", "The report is ready"]
    )
])

print(f"Created category set with {len(basic_categories)} categories:")
for category in basic_categories:
    print(f"  - {category.name}: {category.description[:50]}...")

Created category set with 3 categories:
  - positive: Text expressing positive emotions, satisfaction, o...
  - negative: Text expressing negative emotions, complaints, or ...
  - neutral: Text that is factual or doesn't express strong emo...


In [3]:
# Step 2: Initialize the LLM
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1,  # Low temperature for consistent results
)

# Step 3: Create a classifier
classifier = Classifier(
    llm=llm,
    categories=basic_categories
)

print(f"✅ Classifier initialized: {classifier}")

✅ Classifier initialized: Classifier(DefaultLLMProvider, 3 categories)


In [4]:
# Step 4: Classify some text
sample_texts = [
    "I absolutely love this new feature!",
    "This product is broken and doesn't work",
    "The weather forecast shows rain tomorrow",
    "Best purchase I've ever made!",
    "Customer service was unhelpful and rude",
    "The meeting has been rescheduled to Friday"
]

print("🔍 Basic Classification Results:")
print("=" * 50)

for text in sample_texts:
    result = classifier.classify(text)
    print(f"Text: '{text}'")
    print(f"Category: {result.category}")
    if result.confidence:
        print(f"Confidence: {result.confidence:.2f}")
    print()

🔍 Basic Classification Results:
Text: 'I absolutely love this new feature!'
Category: positive

Text: 'This product is broken and doesn't work'
Category: negative

Text: 'The weather forecast shows rain tomorrow'
Category: neutral

Text: 'Best purchase I've ever made!'
Category: positive

Text: 'Customer service was unhelpful and rude'
Category: negative

Text: 'The meeting has been rescheduled to Friday'
Category: neutral



## 2. Real-World Example: Sentiment Analysis for Customer Feedback

Let's create a more sophisticated sentiment analysis system for customer feedback.

In [5]:
# Define detailed sentiment categories
sentiment_categories = CategorySet(categories=[
    Category(
        name="very_positive",
        description="Extremely positive feedback with high satisfaction, praise, or enthusiasm",
        examples=[
            "Outstanding service! Exceeded all expectations!",
            "Absolutely amazing product, couldn't be happier!",
            "Best experience I've ever had with a company"
        ]
    ),
    Category(
        name="positive",
        description="Generally positive feedback with satisfaction or approval",
        examples=[
            "Good product, works as expected",
            "Happy with the service, would recommend",
            "Nice quality, fair price"
        ]
    ),
    Category(
        name="neutral",
        description="Balanced feedback without strong positive or negative sentiment",
        examples=[
            "Product is okay, nothing special",
            "Average service, met basic expectations",
            "It works fine, no complaints but no praise either"
        ]
    ),
    Category(
        name="negative",
        description="Generally negative feedback with dissatisfaction or complaints",
        examples=[
            "Product didn't work as advertised",
            "Poor customer service, took too long",
            "Not worth the money, disappointed"
        ]
    ),
    Category(
        name="very_negative",
        description="Extremely negative feedback with strong dissatisfaction, anger, or frustration",
        examples=[
            "Terrible experience, complete waste of money!",
            "Worst customer service ever, absolutely horrible!",
            "Product broke immediately, total garbage!"
        ]
    )
])

# Create sentiment classifier
sentiment_classifier = Classifier(
    llm=llm,
    categories=sentiment_categories
)

print(f"✅ Sentiment classifier created with {len(sentiment_categories)} categories")

✅ Sentiment classifier created with 5 categories


In [6]:
# Sample customer feedback data
customer_feedback = [
    "This is hands down the best product I've ever purchased! The quality is incredible and the customer service team went above and beyond to help me. I'm telling all my friends about this!",
    "The product works fine and arrived on time. Nothing to complain about, but nothing extraordinary either.",
    "I'm really disappointed with this purchase. The product broke after just two days and the return process was a nightmare. Would not recommend.",
    "Great value for money! The product exceeded my expectations and the shipping was fast. Very satisfied with my purchase.",
    "Absolutely terrible experience. The product was defective, customer service was rude, and I still haven't received my refund after 3 weeks. Never buying from this company again!",
    "The product is decent. It does what it's supposed to do, though the design could be better. Overall, it's an okay purchase.",
    "I love this product! It's exactly what I needed and the quality is fantastic. The packaging was also very professional.",
    "Not happy with this purchase. The product feels cheap and doesn't work as advertised. Requesting a return."
]

print("📊 Customer Feedback Sentiment Analysis:")
print("=" * 60)

sentiment_counts = {}

for i, feedback in enumerate(customer_feedback, 1):
    result = sentiment_classifier.classify(feedback)
    
    # Count sentiments
    sentiment_counts[result.category] = sentiment_counts.get(result.category, 0) + 1
    
    print(f"Feedback #{i}:")
    print(f"Text: '{feedback[:80]}{'...' if len(feedback) > 80 else ''}'")
    print(f"Sentiment: {result.category.replace('_', ' ').title()}")
    if result.confidence:
        print(f"Confidence: {result.confidence:.2f}")
    print()

# Summary statistics
print("📈 Sentiment Distribution:")
print("-" * 30)
total_feedback = len(customer_feedback)
for sentiment, count in sorted(sentiment_counts.items()):
    percentage = (count / total_feedback) * 100
    print(f"{sentiment.replace('_', ' ').title()}: {count} ({percentage:.1f}%)")

📊 Customer Feedback Sentiment Analysis:
Feedback #1:
Text: 'This is hands down the best product I've ever purchased! The quality is incredib...'
Sentiment: Very Positive

Feedback #2:
Text: 'The product works fine and arrived on time. Nothing to complain about, but nothi...'
Sentiment: Neutral

Feedback #3:
Text: 'I'm really disappointed with this purchase. The product broke after just two day...'
Sentiment: Very Negative

Feedback #4:
Text: 'Great value for money! The product exceeded my expectations and the shipping was...'
Sentiment: Very Positive

Feedback #5:
Text: 'Absolutely terrible experience. The product was defective, customer service was ...'
Sentiment: Very Negative

Feedback #6:
Text: 'The product is decent. It does what it's supposed to do, though the design could...'
Sentiment: Neutral

Feedback #7:
Text: 'I love this product! It's exactly what I needed and the quality is fantastic. Th...'
Sentiment: Very Positive

Feedback #8:
Text: 'Not happy with this purchase. The p

## 3. Topic Classification: News Article Categorization

Let's build a news article classifier to demonstrate topic classification.

In [7]:
# Define news topic categories
news_categories = CategorySet(categories=[
    Category(
        name="technology",
        description="Articles about technology, software, hardware, AI, startups, and tech industry news",
        examples=[
            "Apple announces new iPhone with advanced AI features",
            "Startup raises $50M for revolutionary blockchain platform",
            "Google's latest AI model shows breakthrough performance"
        ]
    ),
    Category(
        name="business",
        description="Business news, finance, economics, markets, corporate news, and economic policy",
        examples=[
            "Stock market reaches new highs amid economic recovery",
            "Major merger announced between two Fortune 500 companies",
            "Federal Reserve announces interest rate changes"
        ]
    ),
    Category(
        name="politics",
        description="Political news, government policy, elections, international relations, and political analysis",
        examples=[
            "President signs new climate change legislation",
            "Congressional hearing on healthcare reform scheduled",
            "International trade agreement reached between nations"
        ]
    ),
    Category(
        name="health",
        description="Health and medical news, disease outbreaks, medical research, healthcare policy, and wellness",
        examples=[
            "New study shows promising results for cancer treatment",
            "WHO announces global health initiative",
            "Breakthrough in Alzheimer's research published"
        ]
    ),
    Category(
        name="sports",
        description="Sports news, game results, player transfers, tournaments, and athletic achievements",
        examples=[
            "Championship game ends in dramatic overtime victory",
            "Star player signs record-breaking contract",
            "Olympic preparations underway for upcoming games"
        ]
    ),
    Category(
        name="entertainment",
        description="Entertainment news, movies, TV shows, music, celebrities, and cultural events",
        examples=[
            "Blockbuster movie breaks box office records",
            "Popular TV series announces final season",
            "Music festival lineup revealed for summer"
        ]
    )
])

# Create news classifier
news_classifier = Classifier(
    llm=llm,
    categories=news_categories
)

print(f"✅ News classifier created with {len(news_categories)} categories")

✅ News classifier created with 6 categories


In [8]:
# Sample news headlines and snippets
news_articles = [
    "Tesla's new autonomous driving software shows 40% improvement in safety metrics during beta testing phase",
    "Federal Reserve raises interest rates by 0.25% to combat inflation, markets respond positively",
    "Senator introduces bipartisan bill to reform immigration policy, gaining support from both parties",
    "Clinical trial results show new Alzheimer's drug slows cognitive decline by 35% in early-stage patients",
    "World Cup final attracts record-breaking 1.5 billion viewers worldwide, highest in tournament history",
    "Marvel announces Phase 5 of cinematic universe with 12 new movies and series over next three years",
    "Quantum computing startup achieves breakthrough in error correction, attracting $100M investment round",
    "Major airline reports 15% increase in quarterly profits as travel demand continues to surge post-pandemic",
    "Supreme Court to hear landmark case on digital privacy rights in the age of social media",
    "New COVID variant detected, health officials monitor transmission rates and vaccine effectiveness"
]

print("📰 News Article Classification:")
print("=" * 50)

topic_counts = {}

for i, article in enumerate(news_articles, 1):
    result = news_classifier.classify(article)
    
    # Count topics
    topic_counts[result.category] = topic_counts.get(result.category, 0) + 1
    
    print(f"Article #{i}:")
    print(f"Headline: '{article}'")
    print(f"Topic: {result.category.title()}")
    if result.confidence:
        print(f"Confidence: {result.confidence:.2f}")
    print()

# Topic distribution
print("📊 Topic Distribution:")
print("-" * 25)
total_articles = len(news_articles)
for topic, count in sorted(topic_counts.items()):
    percentage = (count / total_articles) * 100
    print(f"{topic.title()}: {count} ({percentage:.1f}%)")

📰 News Article Classification:
Article #1:
Headline: 'Tesla's new autonomous driving software shows 40% improvement in safety metrics during beta testing phase'
Topic: Technology

Article #2:
Headline: 'Federal Reserve raises interest rates by 0.25% to combat inflation, markets respond positively'
Topic: Business

Article #3:
Headline: 'Senator introduces bipartisan bill to reform immigration policy, gaining support from both parties'
Topic: Politics

Article #4:
Headline: 'Clinical trial results show new Alzheimer's drug slows cognitive decline by 35% in early-stage patients'
Topic: Health

Article #5:
Headline: 'World Cup final attracts record-breaking 1.5 billion viewers worldwide, highest in tournament history'
Topic: Sports

Article #6:
Headline: 'Marvel announces Phase 5 of cinematic universe with 12 new movies and series over next three years'
Topic: Entertainment

Article #7:
Headline: 'Quantum computing startup achieves breakthrough in error correction, attracting $100M invest

## 4. Content Moderation: Spam and Safety Detection

Let's create a content moderation system to detect spam, inappropriate content, and safety issues.

In [9]:
# Define content moderation categories
moderation_categories = CategorySet(categories=[
    Category(
        name="safe",
        description="Content that is appropriate, helpful, and follows community guidelines",
        examples=[
            "How do I bake a chocolate cake?",
            "What's the weather like today?",
            "Thank you for your help with the project!"
        ]
    ),
    Category(
        name="spam",
        description="Promotional content, advertisements, repetitive messages, or unsolicited marketing",
        examples=[
            "Buy now! Limited time offer! Click here!",
            "Make money fast with this one simple trick!",
            "URGENT: Claim your prize now before it expires!"
        ]
    ),
    Category(
        name="inappropriate",
        description="Content that violates community guidelines, is offensive, or contains inappropriate material",
        examples=[
            "Hate speech or discriminatory language",
            "Harassment or bullying behavior",
            "Explicit or inappropriate content"
        ]
    ),
    Category(
        name="suspicious",
        description="Content that might be misleading, contains misinformation, or requires manual review",
        examples=[
            "Unverified medical claims or advice",
            "Conspiracy theories or misinformation",
            "Suspicious links or phishing attempts"
        ]
    )
])

# Create moderation classifier
moderation_classifier = Classifier(
    llm=llm,
    categories=moderation_categories
)

print(f"✅ Content moderation classifier created with {len(moderation_categories)} categories")

✅ Content moderation classifier created with 4 categories


In [10]:
# Sample content for moderation
content_samples = [
    "Can someone help me understand how to set up a Python virtual environment?",
    "🚨 URGENT! You've won $10,000! Click this link NOW to claim your prize before it expires! Limited time only!!!",
    "I disagree with your opinion, but I respect your right to have it. Let's discuss this civilly.",
    "This group is full of idiots who don't know anything. You're all stupid and should shut up.",
    "Did you know that drinking bleach can cure all diseases? My friend's cousin tried it and it worked!",
    "Thanks for sharing that tutorial! It really helped me solve my coding problem.",
    "MAKE MONEY FAST! Work from home! No experience needed! Earn $5000/week! Contact us now!",
    "I heard that vaccines contain microchips to track people. The government doesn't want you to know this!",
    "What's the best way to learn machine learning for beginners? Any good resources?",
    "Click here to download free software! Definitely not a virus! 100% safe! www.suspicious-site.com"
]

print("🛡️ Content Moderation Results:")
print("=" * 50)

moderation_counts = {}
flagged_content = []

for i, content in enumerate(content_samples, 1):
    result = moderation_classifier.classify(content)
    
    # Count categories
    moderation_counts[result.category] = moderation_counts.get(result.category, 0) + 1
    
    # Track flagged content
    if result.category != "safe":
        flagged_content.append((content, result.category))
    
    # Determine emoji based on category
    emoji_map = {
        "safe": "✅",
        "spam": "🚫",
        "inappropriate": "⚠️",
        "suspicious": "🔍"
    }
    
    print(f"Content #{i}: {emoji_map.get(result.category, '❓')}")
    print(f"Text: '{content[:80]}{'...' if len(content) > 80 else ''}'")
    print(f"Classification: {result.category.title()}")
    if result.confidence:
        print(f"Confidence: {result.confidence:.2f}")
    print()

# Moderation summary
print("📊 Moderation Summary:")
print("-" * 30)
total_content = len(content_samples)
safe_content = moderation_counts.get("safe", 0)
flagged_count = total_content - safe_content

print(f"Total Content: {total_content}")
print(f"Safe Content: {safe_content} ({(safe_content/total_content)*100:.1f}%)")
print(f"Flagged Content: {flagged_count} ({(flagged_count/total_content)*100:.1f}%)")
print()

for category, count in sorted(moderation_counts.items()):
    percentage = (count / total_content) * 100
    print(f"{category.title()}: {count} ({percentage:.1f}%)")

🛡️ Content Moderation Results:
Content #1: ✅
Text: 'Can someone help me understand how to set up a Python virtual environment?'
Classification: Safe

Content #2: 🚫
Text: '🚨 URGENT! You've won $10,000! Click this link NOW to claim your prize before it ...'
Classification: Spam

Content #3: ✅
Text: 'I disagree with your opinion, but I respect your right to have it. Let's discuss...'
Classification: Safe

Content #4: ⚠️
Text: 'This group is full of idiots who don't know anything. You're all stupid and shou...'
Classification: Inappropriate

Content #5: 🔍
Text: 'Did you know that drinking bleach can cure all diseases? My friend's cousin trie...'
Classification: Suspicious

Content #6: ✅
Text: 'Thanks for sharing that tutorial! It really helped me solve my coding problem.'
Classification: Safe

Content #7: 🚫
Text: 'MAKE MONEY FAST! Work from home! No experience needed! Earn $5000/week! Contact ...'
Classification: Spam

Content #8: 🔍
Text: 'I heard that vaccines contain microchips to track 

## 5. Voting Classifier: Improving Accuracy with Ensemble Methods

Now let's explore Tagmatic's powerful voting classifier feature, which can significantly improve classification accuracy.

In [11]:
# Let's use some ambiguous or challenging examples to test voting
challenging_texts = [
    "This product is okay, I guess. Not great but not terrible either.",  # Ambiguous sentiment
    "The new AI model shows promise but still has significant limitations.",  # Mixed sentiment
    "While I appreciate the effort, the execution could be much better.",  # Polite criticism
    "It's fine for the price, though you get what you pay for.",  # Conditional positive
    "Not bad, but I've seen better. Would consider other options next time."  # Lukewarm review
]

print("🗳️ Comparing Single vs Voting Classification:")
print("=" * 60)

for i, text in enumerate(challenging_texts, 1):
    print(f"\nTest Case #{i}:")
    print(f"Text: '{text}'")
    print("-" * 40)
    
    # Single classification
    single_result = sentiment_classifier.classify(text)
    print(f"Single Classification: {single_result.category}")
    
    # Voting classification with 3 rounds
    voting_result_3 = sentiment_classifier.voting_classify(text, voting_rounds=3)
    print(f"Voting (3 rounds): {voting_result_3.category} (confidence: {voting_result_3.confidence:.2f})")
    print(f"  Individual votes: {voting_result_3.votes}")
    
    # Voting classification with 5 rounds
    voting_result_5 = sentiment_classifier.voting_classify(text, voting_rounds=5)
    print(f"Voting (5 rounds): {voting_result_5.category} (confidence: {voting_result_5.confidence:.2f})")
    print(f"  Vote distribution: {voting_result_5.vote_counts}")
    print(f"  Unanimous decision: {voting_result_5.is_unanimous}")
    
    # Analysis
    if voting_result_5.confidence >= 0.8:
        confidence_level = "High"
    elif voting_result_5.confidence >= 0.6:
        confidence_level = "Medium"
    else:
        confidence_level = "Low"
    
    print(f"  Confidence Level: {confidence_level}")
    
    if single_result.category != voting_result_5.category:
        print(f"  ⚠️  Voting changed result: {single_result.category} → {voting_result_5.category}")
    else:
        print(f"  ✅ Consistent result across methods")

🗳️ Comparing Single vs Voting Classification:

Test Case #1:
Text: 'This product is okay, I guess. Not great but not terrible either.'
----------------------------------------
Single Classification: neutral
Voting (3 rounds): neutral (confidence: 1.00)
  Individual votes: ['neutral', 'neutral', 'neutral']
Voting (5 rounds): neutral (confidence: 1.00)
  Vote distribution: {'neutral': 5}
  Unanimous decision: True
  Confidence Level: High
  ✅ Consistent result across methods

Test Case #2:
Text: 'The new AI model shows promise but still has significant limitations.'
----------------------------------------
Single Classification: neutral
Voting (3 rounds): neutral (confidence: 1.00)
  Individual votes: ['neutral', 'neutral', 'neutral']
Voting (5 rounds): neutral (confidence: 1.00)
  Vote distribution: {'neutral': 5}
  Unanimous decision: True
  Confidence Level: High
  ✅ Consistent result across methods

Test Case #3:
Text: 'While I appreciate the effort, the execution could be much bette

## 6. Advanced Features

Tagmatic offers several advanced features for more complex use cases.

## 7. Best Practices for Optimal Performance

To get the best results from Tagmatic:

1.  **Clear and Distinct Categories**: 
    *   Ensure your category names are concise and descriptions are unambiguous.
    *   Avoid overlapping categories as much as possible. If overlap is unavoidable, ensure descriptions clearly differentiate them.

2.  **High-Quality Examples**:
    *   Provide diverse and representative examples for each category.
    *   Include edge cases or tricky examples if known.
    *   The quality of examples significantly impacts LLM performance.

3.  **Iterate on Categories and Prompts**:
    *   Start with a basic set of categories and refine them based on classification results.
    *   Experiment with prompt phrasing if the default prompt isn't yielding optimal results for your specific LLM or task.

4.  **Use Voting for Critical Tasks**:
    *   For applications requiring high accuracy, leverage the `voting_classify` method.
    *   Balance the number of voting rounds with cost and latency considerations.

5.  **Choose the Right LLM**:
    *   Different LLMs have varying strengths. Experiment with models available through LangChain (e.g., GPT-3.5-turbo, GPT-4, Claude, etc.) to find the best fit for your budget and performance needs.
    *   More powerful models might provide better accuracy but at a higher cost.

6.  **Temperature Setting**:
    *   For classification, use a low temperature (e.g., 0.0 to 0.2) to ensure consistent and deterministic outputs.

7.  **Pre-processing Text (Optional)**:
    *   Depending on your data, pre-processing steps like lowercasing, removing special characters, or handling HTML tags might be beneficial, though modern LLMs are often robust to raw text.

8.  **Monitor and Evaluate**:
    *   Regularly evaluate the classifier's performance on a validation set.
    *   Track metrics like accuracy, precision, recall, and F1-score for each category.
    *   Use tools like confusion matrices to identify common misclassifications.

9.  **Consider Cost and Latency**:
    *   LLM API calls have associated costs and latencies. Optimize your usage by using batch processing where appropriate and choosing models that meet your budget and speed requirements.

10. **Error Handling and Fallbacks**:
    *   Implement robust error handling for API calls.
    *   Consider fallback mechanisms or human review for low-confidence classifications or critical decisions.

## Conclusion

Congratulations on completing the Tagmatic tutorial! 🎉

You've learned how to:
- Set up Tagmatic and define categories.
- Perform basic text classification.
- Implement real-world use cases like sentiment analysis, topic classification, and content moderation.
- Improve accuracy using the voting classifier.
- Utilize advanced features like batch processing and custom prompts.
- Apply best practices for optimal performance.

Tagmatic provides a flexible and powerful way to leverage LLMs for your text classification needs. We encourage you to experiment further, explore different LLMs, and integrate Tagmatic into your projects.


Happy classifying!